#Install Packages

In [ ]:
import sys
import subprocess
import pkg_resources
#!pip install -Uqq ipdb


required = {
      'kaggle'
      , 'google-cloud-aiplatform'
      , 'smart-open'
      , 'ipdb'
    }

print(f"working set entries: {pkg_resources.working_set}")

specific_required = set()

for pkg in required:
  if not '==' in pkg and pkg in pkg_resources.working_set.by_key.keys():
    str_key=str( pkg_resources.get_distribution(pkg) ).replace(' ','==')
    print(f"adding {str_key}")
    specific_required.add(str_key)
  else:
    specific_required.add(pkg)

required=specific_required

required.discard(None)
installed = { f"{pkg.key}=={pkg_resources.get_distribution(pkg.key).version}" for pkg in pkg_resources.working_set}
missing = required - installed
print(f"missing: {missing}")
if missing:
    python = sys.executable
    #subprocess.check_call([python, '-m', 'pip', 'install', '--upgrade', f"--target={PYTHON_LIB_PATH}", *missing], stdout=subprocess.DEVNULL)
    subprocess.check_call([python, '-m', 'pip', 'install', '--upgrade', *missing], stdout=subprocess.DEVNULL)

    import IPython
    import time
    print( 'restarting kernel...' )
    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

    # Wait for the kernel to "crash". Haven't found a more elegant workaround for do_shutdown being an async call...

working set entries: <pkg_resources.WorkingSet object at 0x7f0f90f45070>
adding kaggle==1.5.13
adding google-cloud-aiplatform==1.23.0
adding smart-open==6.3.0
adding ipdb==0.13.13
missing: set()


In [ ]:
import sys
if 'google.colab' in sys.modules:
  from google.colab import auth
  auth.authenticate_user()
  print('Authenticated')


import google.auth
credentials, project = google.auth.default()
from google.cloud.bigquery import magics
magics.context.credentials = credentials

Authenticated


In [ ]:
PROJECT_ID = "kallogjeri-project-345114" # @param {type:"string"} <---CHANGE THESE
KAGGLE_DATASET_USERNAME = "sorkun" # @param {type:"string"} <---CHANGE THESE
KAGGLE_DATASET_NAME = "aqsoldb-a-curated-aqueous-solubility-dataset" # @param {type:"string"} <---CHANGE THESE
BUCKET_NAME = "medical_text_demo" # @param {type:"string"} <---CHANGE THESE
GOOGLE_APPLICATION_CREDENTIALS="/content/service_account.json"
WORKSPACE_HOME="/content" # @param {type:"string"} <---CHANGE THESE

import sys
IN_COLAB = 'google.colab' in sys.modules
%env IN_COLAB=$IN_COLAB

%env KAGGLE_DATASET_USERNAME = KAGGLE_DATASET_USERNAME
%env KAGGLE_DATASET_NAME = KAGGLE_DATASET_NAME
%env GOOGLE_APPLICATION_CREDENTIALS=$GOOGLE_APPLICATION_CREDENTIALS
%env BUCKET_NAME=$BUCKET_NAME
!gcloud config set project $PROJECT_ID -q

env: IN_COLAB=True
env: KAGGLE_DATASET_USERNAME=KAGGLE_DATASET_USERNAME
env: KAGGLE_DATASET_NAME=KAGGLE_DATASET_NAME
env: GOOGLE_APPLICATION_CREDENTIALS=/content/service_account.json
env: BUCKET_NAME=medical_text_demo
Updated property [core/project].


In [ ]:
import sys
import os

FILE_EXISTS=os.path.exists( '/content/service_account.json' )
IN_COLAB = 'google.colab' in sys.modules
%env IN_COLAB=$IN_COLAB

if IN_COLAB and not FILE_EXISTS:
  from google.colab import files

  uploaded = files.upload()

  for fn in uploaded.keys():
    txt = "User uploaded file {name} with length {length} bytes".format( name=fn, length=len(uploaded[fn]))
    print( txt )

  import os
  os.rename(f"/content/{fn}",'/content/service_account.json')

env: IN_COLAB=True


Saving kallogjeri-project-345114@appspot.gserviceaccount.com.json to kallogjeri-project-345114@appspot.gserviceaccount.com.json
User uploaded file kallogjeri-project-345114@appspot.gserviceaccount.com.json with length 2336 bytes


Upload Kaggle .json file

In [ ]:
import sys
import os

FILE_EXISTS=os.path.exists( '/root/.kaggle/kaggle.json' )
if IN_COLAB and not FILE_EXISTS:

  !mkdir -p /root/.kaggle

  from google.colab import files

  uploaded = files.upload()

  for fn in uploaded.keys():
    txt = "User uploaded file {name} with length {length} bytes".format( name=fn, length=len(uploaded[fn]))
    print( txt )

  import os
  os.rename(f"{fn}",'/root/.kaggle/kaggle.json')
  !chmod 600 /root/.kaggle/kaggle.json

  # import json
  # token = {"username":"alexburdenko","key":"3eb670cc106e7b8cfe41dfeaab51e3a2"}
  # with open('/root/.kaggle/kaggle.json', 'w') as file:
  #   json.dump(token, file)


!cat $HOME/.kaggle/kaggle.json

# %env KAGGLE_USERNAME = 'alexburdenko'
# %env KAGGLE_KEY = '0503be621facfe2634edfc421af8d8e0'


Saving kaggle.json to kaggle.json
User uploaded file kaggle.json with length 68 bytes
{"username":"alexburdenko","key":"bdc6f69f4aa714222d8c52fbf1aeca28"}

In [ ]:
!kaggle competitions list

ref                                                                                deadline             category             reward  teamCount  userHasEntered  
---------------------------------------------------------------------------------  -------------------  ---------------  ----------  ---------  --------------  
https://www.kaggle.com/competitions/vesuvius-challenge-ink-detection               2023-06-14 23:59:00  Featured         $1,000,000        304           False  
https://www.kaggle.com/competitions/asl-signs                                      2023-05-01 23:59:00  Research           $100,000        840           False  
https://www.kaggle.com/competitions/tlvmc-parkinsons-freezing-gait-prediction      2023-06-08 23:59:00  Research           $100,000        348           False  
https://www.kaggle.com/competitions/amp-parkinsons-disease-progression-prediction  2023-05-18 23:59:00  Featured            $60,000        966           False  
https://www.kaggle.com/competition

In [ ]:
!kaggle datasets download $KAGGLE_DATASET_USERNAME/$KAGGLE_DATASET_NAME --force

  0% 0.00/1.39M [00:00<?, ?B/s]
100% 1.39M/1.39M [00:00<00:00, 66.3MB/s]


# Create Vertex Dataset

In [ ]:
!gsutil -m cp -r /content/{KAGGLE_DATASET_NAME}.zip gs://{BUCKET_NAME}
!gsutil ls -al gs://{BUCKET_NAME}

Copying file:///content/aqsoldb-a-curated-aqueous-solubility-dataset.zip [Content-Type=application/zip]...
\
Operation completed over 1 objects/1.4 MiB.                                      
   1454326  2023-04-04T15:48:20Z  gs://medical_text_demo/aqsoldb-a-curated-aqueous-solubility-dataset.zip#1680623300803909  metageneration=1
                                 gs://medical_text_demo/bq_import/
                                 gs://medical_text_demo/docai_processed/
                                 gs://medical_text_demo/hcls_nl_json/
                                 gs://medical_text_demo/jsonl/
                                 gs://medical_text_demo/medical-text.zip/
                                 gs://medical_text_demo/pdf/
                                 gs://medical_text_demo/raw_text/
                                 gs://medical_text_demo/train/
TOTAL: 1 objects, 1454326 bytes (1.39 MiB)


In [ ]:
from google.cloud import storage
from zipfile import ZipFile
from zipfile import is_zipfile
import io

def zipextract(bucketname, zipfilename_with_path):

    storage_client = storage.Client()
    bucket = storage_client.get_bucket(bucketname)

    destination_blob_pathname = zipfilename_with_path

    blob = bucket.blob(destination_blob_pathname)
    zipbytes = io.BytesIO(blob.download_as_string())

    if is_zipfile(zipbytes):
        with ZipFile(zipbytes, 'r') as myzip:
            for contentfilename in myzip.namelist():
                contentfile = myzip.read(contentfilename)
                blob2 = bucket.blob(zipfilename_with_path + "/" + contentfilename)
                blob2.upload_from_string(contentfile)

    blob.delete()

filename = f"{KAGGLE_DATASET_NAME}.zip"
zipextract( BUCKET_NAME, filename ) # if the file is gs://mybucket/path/file.zip

In [ ]:
uri=f"gs://{BUCKET_NAME}/{KAGGLE_DATASET_NAME}.zip"

import re
matches = re.match(r"gs://(.*?)/(.*)", uri)

bucket_name = None
folder_path = None
if matches:
  bucket_name, folder_path = matches.groups()


print(f"uri : {uri}")
print(f"Output bucket: {bucket_name}" )
print(f"Output prefix: {folder_path}" )

from google.cloud import storage
storage_client = storage.Client()
blobs = list(storage_client.list_blobs(bucket_name, prefix=f"{folder_path}/", fields="items(name)"))

raw_text_file_path = ''
hcls_nl_json_uri = ''
file_list = list()
file_prefixes = list()
file_map = dict()
for blob in blobs:

  if not blob.name.endswith('/'):
    # Download JSON File as bytes object and convert to Document Object
    file_path=f"gs://{BUCKET_NAME}/{blob.name}"
    file_prefixes.append(f"jsonl/{blob.name}.jsonl")
    print(f"jsonl/{blob.name}.jsonl")
    file_list.append( file_path )

    file_map[blob.name] = f"jsonl/{blob.name}.jsonl"


uri : gs://medical_text_demo/aqsoldb-a-curated-aqueous-solubility-dataset.zip
Output bucket: medical_text_demo
Output prefix: aqsoldb-a-curated-aqueous-solubility-dataset.zip
jsonl/aqsoldb-a-curated-aqueous-solubility-dataset.zip/curated-solubility-dataset.csv.jsonl


In [ ]:
def to_jsonl( annotations : list):
  # import json
  # jsonl_output = ""
  # for annotation in annotations:
  #   jsonl_output += json.dumps(annotation) + "\n"
  # return jsonl_output

  import json
  from io import StringIO

  res = [json.dumps(annotation) + "\n" for annotation in annotations]
  buf = StringIO()
  for i in res:
    buf.write(i)
      #buf.write(i+'\n')

  sbuf = buf.getvalue().replace('"-','\"-')
  sbuf = buf.getvalue().replace("'-","-")
  sbuf = buf.getvalue().replace("'","\"")
  sbuf = buf.getvalue().replace("#","\#")

  print(f"Created import file based on {len(annotations)} annotations")
  return sbuf






In [ ]:
def upload_str_to_bucket(str_buf, bucket_name, file_path = '/' ):
    print("Attempting to upload str to bucket {} and path {}".format( bucket_name, file_path))

    client = storage.Client()
    bucket = client.get_bucket(bucket_name)
    blob: Blob = bucket.blob(file_path)
    blob.cache_control = "no-cache,max-age=0"
    blob.content_encoding = "utf-16"

    blob.upload_from_string( str_buf )

In [ ]:
# def get_text_annotations( full_text: str  ) -> list:
#   annotations = []
#   text_segment_annotations = []
#   annotation = {
#         #"textSegmentAnnotations": text_segment_annotations,
#         "textContent": full_text
#     }

#   annotations.append(annotation)
#   return annotations


In [ ]:
def get_text_annotations( resp )->list:
  #print(resp)
  entity_keys = list()
  automl_entities = list()
  for entity_mention in resp.get('entityMentions', []):
      entity_text = entity_mention['text'].get('content', '') # Insulin regimen human
      entity_type = entity_mention['type']

      start_offset = entity_mention['text'].get('beginOffset', -1)

      ln = len(entity_text)
      end_offset = start_offset + ln

      #highlight_text = raw_text[start_offset:end_offset]

      #print(f"highlight_text: {highlight_text}")
      # print(f"entity_type: {entity_type}")
      # print(f"entity_text: {entity_text}")

      if (start_offset, end_offset) not in entity_keys:

          if start_offset > 0 :
              automl_entity=dict()
              automl_entity['endOffset']=end_offset
              automl_entity['startOffset']=start_offset
              automl_entity['displayName']=entity_type
              #print(automl_entity)
              #print(len(automl_entity))
              automl_entities.append( automl_entity )
              print(f"Start offset: {start_offset}")
              print(f"End offset: {end_offset}")
              entity_keys.append( (start_offset, end_offset) )



      return automl_entities

In [ ]:
from google.cloud import _http
class Connection(_http.JSONConnection):
  """Handles HTTP requests to GCP."""
  API_BASE_URL = 'https://healthcare.googleapis.com'
  API_VERSION = 'v1beta1'
  #API_VERSION = 'v1'
  API_URL_TEMPLATE = '{api_base_url}/{api_version}/projects{path}'

from google.cloud.client import ClientWithProject
class Client(ClientWithProject):
  """A client for accessing Cloud Healthcare NLP API.

  Args:
      project (Union[str, None]): The ID of the project
      region (str): The region the project resides in, e.g. us-central1,
  """

  def __init__(self,
               project,
               region='us-central1',
               credentials=None,
               http=None):
    self.region = region
    self.SCOPE = ('https://www.googleapis.com/auth/cloud-healthcare',)
    super(Client, self).__init__(project=project)
    self.path = '/{}/locations/{}/services/nlp'.format(self.project,
                                                       self.region)
    # self._credentials = credentials
    self.project=project
    self._connection = Connection(self)


  def analyze_entities(self, document):
    """ Analyze the clinical entities a document with the Google Cloud

      Healthcare NLP API.

      Args:
        document (str): the medical document to analyze.

      Returns:
        Dict[str, Any]: the JSON response.
      """

    # client = language.LanguageServiceClient()
    # document = language.Document(content=document, type_=language.Document.Type.PLAIN_TEXT)


    # #document = language.Document(content=document, type = language.Document.Type.PLAIN_TEXT)
    # #document = language.Document(content=document, type_=language.Document.Type.PLAIN_TEXT)

    # client = language.LanguageServiceClient()
    # response = client.analyze_entities(document=document)


    # print( document )
    document = str( document.encode('utf-8') )

    return self._connection.api_request(
        'POST',
        self.path + ':analyzeEntities',
        data={'document_content': document})

def get_one_mb_str_blocks( str_item: str, blocks: list ):
  #MAX_LEN=1000000
  MAX_LEN=20000
  str_len = len( str_item.encode('utf-16') )

  if str_len > MAX_LEN :
     rest = str_item[MAX_LEN+1:]

     if len(rest.encode('utf-16')) > MAX_LEN:
        get_one_mb_str_blocks( rest, blocks )
     else:
        blocks.append( rest )


  else:
      blocks.append( str_item )

def analyze_entities(str_item, project, creds)->dict:
   #API Limits the size of the text field to 1 MB
  str_blocks = list()
  #str_item = str_item.encode(encoding='utf-8')
  get_one_mb_str_blocks( str_item, str_blocks )

  client = Client( project=project, credentials=creds )
  if len(str_blocks) > 1:
    last_fragment = str_blocks[len(str_blocks)-1]

  res = dict()

  for block in str_blocks:

    # str_item = "Sepsis results in unfettered inflammation, tissue damage, and multiple organ failure. Diffuse brain dysfunction and neurological manifestations secondary to sepsis are termed sepsis-associated encephalopathy (SAE). Extracellular nucleotides, proinflammatory cytokines, and oxidative stress reactions are associated with delirium and brain injury, and might be linked to the pathophysiology of SAE. P2X7 receptor activation by extracellular ATP leads to maturation and release of IL-1β by immune cells, which stimulates the production of oxygen reactive species. Hence, we sought to investigate the role of purinergic signaling by P2X7 in a model of sepsis. We also determined how this process is regulated by the ectonucleotidase CD39, a scavenger of extracellular nucleotides. Wild type (WT), P2X7 receptor (P2X7)"
    res.update( client.analyze_entities( block ) )

  return res

In [ ]:
%pdb off
import ipdb
def download_str_from_bucket(bucket_name, file_path = '/default.csv' )->str:
    from google.cloud import storage
    import os
    import pandas as pd

    client = storage.Client()
    bucket = client.get_bucket(bucket_name)
    print(f"checking if {file_path} exists...")
    blob = bucket.blob(file_path)
    exists = blob.exists( client )
    print( exists )

    from smart_open import open
    from io import StringIO
    content_buf = StringIO()


    content = None
    if exists:
      blob_path = f"gs://{bucket_name}/{file_path}"
      print(f"downloading file...{blob_path}")

      for line in open(blob_path, encoding="ISO-8859-1"):
        content_buf.write( line )

      content_buf.flush()
      content = content_buf.getvalue()

    return content

import pprint

import pickle
from os.path import exists

#file_prefixes = [f"gs://{BUCKET_NAME}/jsonl/{prefix}" for prefix in file_prefixes]
for path_prefix in file_map.keys():
  print(bucket_name)
  print(path_prefix)

  file_name = path_prefix.split('/')[-1]
  file_path = f"{WORKSPACE_HOME}/{file_name}_metadata.pickle"
  print(file_path)

  if not exists(file_path):

      content = download_str_from_bucket( bucket_name=bucket_name, file_path=path_prefix )
      with open(file_path, 'wb') as f:
          pickle.dump(content, f)
  else:
      with open(file_path, 'rb') as handle:
          content = pickle.load(handle)


  from google.oauth2 import service_account
  creds = service_account.Credentials.from_service_account_file( GOOGLE_APPLICATION_CREDENTIALS )

  scoped_credentials = creds.with_scopes(
      ['https://www.googleapis.com/auth/cloud-platform'])


  res = analyze_entities( str_item=content, creds=creds, project=PROJECT_ID )




  #text_entities = list()
  #text_annotations = dict()
  #text_annotations['text_segment_annotations'] =  self._get_text_annotations( res )
  #text_annotations['textContent'] = content
  #text_entities.append( text_annotations )

  text_entities = list()
  text_annotations = dict()
  text_annotations['textSegmentAnnotations'] = get_text_annotations( res )
  text_annotations['textContent'] = content
  text_entities.append( text_annotations )

  # from pprint import pprint
  # pprint(text_annotations)


  jsonl = to_jsonl( text_entities )
  print(f"Uploading jsonl file {file_map[path_prefix]}")

# #  ipdb.set_trace(context=6)

  upload_str_to_bucket(f"{jsonl}"
                      , bucket_name=BUCKET_NAME
                      , file_path=f"{file_map[path_prefix]}" )



Automatic pdb calling has been turned OFF
medical_text_demo
aqsoldb-a-curated-aqueous-solubility-dataset.zip/curated-solubility-dataset.csv
/content/curated-solubility-dataset.csv_metadata.pickle
checking if aqsoldb-a-curated-aqueous-solubility-dataset.zip/curated-solubility-dataset.csv exists...
True
downloading file...gs://medical_text_demo/aqsoldb-a-curated-aqueous-solubility-dataset.zip/curated-solubility-dataset.csv
Created import file based on 1 annotations
Uploading jsonl file jsonl/aqsoldb-a-curated-aqueous-solubility-dataset.zip/curated-solubility-dataset.csv.jsonl
Attempting to upload str to bucket medical_text_demo and path jsonl/aqsoldb-a-curated-aqueous-solubility-dataset.zip/curated-solubility-dataset.csv.jsonl


###TODO: create a jsonl file with the CSV data

In [ ]:
from datetime import datetime

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

import google.cloud.aiplatform as aip
from google.cloud.aiplatform.datasets.text_dataset import TextDataset
dataset : TextDataset = aip.TextDataset.create (
    display_name=KAGGLE_DATASET_NAME + "_" + TIMESTAMP,
    gcs_source= map( lambda x: f"gs://{BUCKET_NAME}/{x}", (file_map.values()) ),
    import_schema_uri=aip.schema.dataset.ioformat.text.extraction
)

print(dataset.resource_name)

Creating TextDataset


INFO:google.cloud.aiplatform.datasets.dataset:Creating TextDataset


Create TextDataset backing LRO: projects/273872083706/locations/us-central1/datasets/8567278055398047744/operations/8739725252940005376


INFO:google.cloud.aiplatform.datasets.dataset:Create TextDataset backing LRO: projects/273872083706/locations/us-central1/datasets/8567278055398047744/operations/8739725252940005376


TextDataset created. Resource name: projects/273872083706/locations/us-central1/datasets/8567278055398047744


...

Import TextDataset data backing LRO: projects/273872083706/locations/us-central1/datasets/8567278055398047744/operations/3797024661900886016


INFO:google.cloud.aiplatform.datasets.dataset:Import TextDataset data backing LRO: projects/273872083706/locations/us-central1/datasets/8567278055398047744/operations/3797024661900886016


TextDataset data imported. Resource name: projects/273872083706/locations/us-central1/datasets/8567278055398047744


INFO:google.cloud.aiplatform.datasets.dataset:TextDataset data imported. Resource name: projects/273872083706/locations/us-central1/datasets/8567278055398047744


projects/273872083706/locations/us-central1/datasets/8567278055398047744
